# Test runtime
### Test with IK checkers

**grasp sample 10**  
solve-mean: 1175.2 ms  
solve-median: 740.8 ms  
success rate: 98.0 %  

**grasp sample 20**  
solve-mean: 1190.6 ms  
solve-median: 778.4 ms  
success rate: 96.5 %  

**grasp sample 30**  
solve-mean: 1323.0 ms  
solve-median: 837.4 ms  
success rate: 99.0 %  

### Test without IK checkers

**grasp sample 10**  
solve-mean: 1998.9 ms  
solve-median: 1241.1 ms  
success rate: 88.0 %  

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt
from pkg.controller.combined_robot import *

# params

In [2]:
ROBOT_TYPE = RobotType.panda

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    
else:
    raise(NotImplementedError("Case not defined for {}".format(ROBOT_TYPE)))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [3]:
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [4]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.72,1.52,0.01), (0.25,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.1,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(np.pi/2,0,np.pi/6), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

box2 = gscene.create_safe(GEOTYPE.BOX, "box2", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(np.pi,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

box1.permute_axis_match_z_link()
box2.permute_axis_match_z_link()

##### create PlanningScene

In [5]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [6]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [7]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=None)

In [8]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.SPHERE, TOOL_NAME, link_name=TOOL_LINK, 
                                dims=(0.01,0.01,0.01), center=TOOL_XYZ, rpy=(-np.pi/2,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=False, fixed=True, collision=False)

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
obs_obj = pscene.create_subject(oname="box2", gname="box2", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box2, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box2, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box2, [0,0,-0.026], [0,0,0])})

## Pipeline

In [12]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [13]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [14]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 'floor')


## import planners

In [15]:

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)

checkers_all = [rcheck, gcheck]

# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]

# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

## Test pybullet

In [16]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))

In [17]:
from convert_pscene import *
from plan_rnb import *

#### load pybullet, convert scene

In [18]:
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)
robot_body, body_names, movable_bodies = pscene_to_pybullet(
    pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
print('Objects:', body_names)
saver = WorldSaver()

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
('Objects:', {1L: 'goal', 2L: 'floor', 3L: 'wall', 4L: 'box1', 5L: 'box2'})


### Test with IK checkers

In [19]:
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)

In [20]:
log_wFeas = []
for _ in range(20):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=checkers_all,
                                          tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF, max_time=10)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_wFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p0), ('AtPose', 4L, p0), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p0, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p1), ('AtPose', 5L, p1), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p1, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q0), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-pose-pose]
Setting negate=True for stream [test-cfree-approach-pose]

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.089
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.184
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g1), sample-pose:(4, 1)->(#p1), inverse-kinematics:(4, #p1, #g1)->(#q3, #t74), test-cfree-traj-pose:(#t74, 5, p4)->(), inverse-kinematics:(4, p3, #g1)->(#q2, #t73), test-cfree-traj-pose:(#t73, 4, p3)->(), test-cfree-traj-pose:(#t73, 5, p4)->(), test-cfree-pose-pose:(4, #p1, 5, p4)->(), test-cfree-approach-pose:(4, p3, #g1, 5, p4)->(), test-cfree-approach-pose:(4, #p1, #g1, 5, p4)->(), plan-free-motion:(q4, #q2)->(#t115), plan-free-motion:(#q3, q4)->(#t117), plan-holding-motion:(#q2, #q3, 4, #g1)->(#t116)]
Action plan (5, 0.000): [move_free(q4, #q2, #t115), pick(4, p3, #g1, #q2, #t73), move_holding(#q2, #q3, 4, #g1, #t116), place(4, #p1, #g1, #q3, #t74), move_free(#q3, q4, #t117)]
iter=0, outs=1) sample-grasp:(4)->[(g1)]
iter=0, outs=1) sample-pose:(4, 1)->[(p5)]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_

iter=inf, outs=1) plan-free-motion:(q7, q10)->[(c12)]
iter=inf, outs=1) plan-free-motion:(q8, q7)->[(c13)]
iter=inf, outs=1) plan-holding-motion:(q10, q8, 4, g2)->[(c14)]
Summary: {complexity: 2, cost: 0.000, evaluations: 36, iterations: 3, length: 2, run_time: 0.876, sample_time: 0.660, search_time: 0.217, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 7822 | p_success: 0.965 | overhead: 0.001
External: sample-grasp | n: 9887 | p_success: 0.989 | overhead: 0.002
External: inverse-kinematics | n: 37630 | p_success: 0.183 | overhead: 0.045
External: plan-free-motion | n: 6136 | p_success: 0.894 | overhead: 0.123
External: plan-holding-motion | n: 2383 | p_success: 0.991 | overhead: 0.058
External: test-cfree-pose-pose | n: 3298 | p_success: 1.000 | overhead: 0.001
External: test-cfree-approach-pose | n: 6485 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 12081 | p_success: 0.939 | overhead: 0.001


iter=inf, outs=1) plan-free-motion:(q11, q12)->[(c17)]
iter=inf, outs=1) plan-free-motion:(q14, q11)->[(c18)]
iter=inf, outs=1) plan-holding-motion:(q12, q14, 4, g3)->[(c19)]
Summary: {complexity: 2, cost: 0.000, evaluations: 38, iterations: 3, length: 2, run_time: 0.873, sample_time: 0.576, search_time: 0.297, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 7824 | p_success: 0.965 | overhead: 0.001
External: sample-grasp | n: 9888 | p_success: 0.989 | overhead: 0.002
External: inverse-kinematics | n: 37633 | p_success: 0.183 | overhead: 0.045
External: plan-free-motion | n: 6138 | p_success: 0.894 | overhead: 0.123
External: plan-holding-motion | n: 2384 | p_success: 0.991 | overhead: 0.058
External: test-cfree-pose-pose | n: 3299 | p_success: 1.000 | overhead: 0.001
External: test-cfree-approach-pose | n: 6487 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 12084 | p_success: 0.939 | overhead: 0.

Wrote: statistics/py2/kuka-tamp.pkl
         1675193 function calls (1608259 primitive calls) in 1.214 seconds

   Ordered by: internal time
   List reduced from 1508 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.528    0.176    0.531    0.177 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
   106430    0.068    0.000    0.087    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.054    0.027    0.219    0.109 /usr/lib/python2.7/pickle.py:851(load)
  53295/1    0.044    0.000    0.110    0.110 /usr/lib/python2.7/pickle.py:269(save)
   213136    0.039    0.000    0.039    0.000 {method 'read' of 'file' objects}
   106398    0.034    0.000    0.046    0.000 /usr/lib/python2.7/pickle.py:1184(load_append)
        2    0.024    0.012    0.024    0.012 {posix.read}
    53230    0.018    0.000    0.030    0.000 /usr/lib/python2.7/pickle.py:443(save_int)
   106588    0.017    0.000    0.017  

...

Wrote: statistics/py2/kuka-tamp.pkl
         1680414 function calls (1613348 primitive calls) in 1.384 seconds

   Ordered by: internal time
   List reduced from 1507 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.792    0.264    0.792    0.264 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
   106790    0.056    0.000    0.073    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.047    0.024    0.186    0.093 /usr/lib/python2.7/pickle.py:851(load)
  53476/1    0.039    0.000    0.093    0.093 /usr/lib/python2.7/pickle.py:269(save)
   213856    0.036    0.000    0.036    0.000 {method 'read' of 'file' objects}
   106758    0.027    0.000    0.037    0.000 /usr/lib/python2.7/pickle.py:1184(load_append)
        2    0.026    0.013    0.026    0.013 {posix.read}
   106950    0.015    0.000    0.015    0.000 {method 'write' of 'file' objects}
    53411    0.015    0.000    0.025    0.000 

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.074
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.164
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g15), inverse-kinematics:(4, p64, #g15)->(#q36, #t2337), test-cfree-traj-pose:(#t2337, 5, p65)->(), sample-pose:(4, 1)->(#p17), inverse-kinematics:(4, #p17, #g15)->(#q37, #t2338), test-cfree-traj-pose:(#t2338, 5, p65)->(), test-cfree-traj-pose:(#t2337, 4, p64)->(), test-cfree-pose-pose:(4, #p17, 5, p65)->(), test-cfree-approach-pose:(4, p64, #g15, 5, p65)->(), test-cfree-approach-pose:(4, #p17, #g15, 5, p65)->(), plan-free-motion:(q137, #q36)->(#t2380), plan-free-motion:(#q37, q137)->(#t2381), plan-holding-motion:(#q36, #q37, 4, #g15)->(#t2379)]
Action plan (5, 0.000): [move_free(q137, #q36, #t2380), pick(4, p64, #g15, #q36, #t2337), move_holding(#q36, #q37, 4, #g15, #t2379), place(4, #p17, #g15, #q37, #t2338), move_free(#q37, q137, #t2381)]
iter=0, outs=1) sample-grasp:(4)->[(g29)]
q_approach: None
obstac

iter=inf, outs=1) plan-free-motion:(q141, q140)->[(c97)]
iter=inf, outs=1) plan-free-motion:(q140, q142)->[(c98)]
iter=inf, outs=1) plan-holding-motion:(q142, q141, 4, g30)->[(c99)]
Summary: {complexity: 2, cost: 0.000, evaluations: 38, iterations: 3, length: 2, run_time: 0.872, sample_time: 0.646, search_time: 0.226, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 7856 | p_success: 0.965 | overhead: 0.001
External: sample-grasp | n: 9915 | p_success: 0.989 | overhead: 0.002
External: inverse-kinematics | n: 37767 | p_success: 0.183 | overhead: 0.045
External: plan-free-motion | n: 6167 | p_success: 0.894 | overhead: 0.124
External: plan-holding-motion | n: 2398 | p_success: 0.991 | overhead: 0.059
External: test-cfree-pose-pose | n: 3313 | p_success: 1.000 | overhead: 0.001
External: test-cfree-approach-pose | n: 6516 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 12139 | p_success: 0.940 | overh

         1684150 function calls (1617006 primitive calls) in 1.109 seconds

   Ordered by: internal time
   List reduced from 1508 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.477    0.159    0.478    0.159 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
   106902    0.061    0.000    0.077    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.049    0.025    0.194    0.097 /usr/lib/python2.7/pickle.py:851(load)
  53531/1    0.039    0.000    0.095    0.095 /usr/lib/python2.7/pickle.py:269(save)
   214080    0.035    0.000    0.035    0.000 {method 'read' of 'file' objects}
   106870    0.030    0.000    0.041    0.000 /usr/lib/python2.7/pickle.py:1184(load_append)
        2    0.027    0.014    0.027    0.014 {posix.read}
    53466    0.016    0.000    0.025    0.000 /usr/lib/python2.7/pickle.py:443(save_int)
   107060    0.014    0.000    0.014    0.000 {method 'write' of 'file' ob

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.079
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.180
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g19), sample-pose:(4, 1)->(#p21), inverse-kinematics:(4, #p21, #g19)->(#q45, #t2574), test-cfree-traj-pose:(#t2574, 5, p79)->(), inverse-kinematics:(4, p78, #g19)->(#q44, #t2573), test-cfree-traj-pose:(#t2573, 4, p78)->(), test-cfree-traj-pose:(#t2573, 5, p79)->(), test-cfree-pose-pose:(4, #p21, 5, p79)->(), test-cfree-approach-pose:(4, p78, #g19, 5, p79)->(), test-cfree-approach-pose:(4, #p21, #g19, 5, p79)->(), plan-free-motion:(#q45, q149)->(#t2616), plan-free-motion:(q149, #q44)->(#t2617), plan-holding-motion:(#q44, #q45, 4, #g19)->(#t2615)]
Action plan (5, 0.000): [move_free(q149, #q44, #t2617), pick(4, p78, #g19, #q44, #t2573), move_holding(#q44, #q45, 4, #g19, #t2615), place(4, #p21, #g19, #q45, #t2574), move_free(#q45, q149, #t2616)]
iter=0, outs=1) sample-grasp:(4)->[(g36)]
iter=0, outs=1) sample-

In [21]:
print(gtimer)

time_array = np.array(gtimer.timelist_dict["solve"])
success_array = np.array(log_wFeas)<1
print("solve-mean: {} ms".format(np.round(np.mean(time_array[np.where(success_array)[0]]), 1)))
print("solve-median: {} ms".format(np.round(np.median(time_array[np.where(success_array)[0]]), 1)))
print("success rate: {} %".format(np.mean(success_array)*100))

solve: 	29848.4 ms/20 = 1492.4 ms (942.475/6524.864)
sample_grasps_4: 	87.3 ms/20 = 4.4 ms (2.348/9.821)
get_stable_4_1: 	25.2 ms/34 = 0.7 ms (0.433/1.575)
ik_fn: 	3560.2 ms/154 = 23.1 ms (0.899/165.385)
check_feas: 	506.1 ms/154 = 3.3 ms (0.882/8.068)
ReachChecker: 	125.0 ms/154 = 0.8 ms (0.339/2.193)
GraspChecker: 	271.3 ms/148 = 1.8 ms (0.657/6.068)
free_motion_gen: 	9458.8 ms/43 = 220.0 ms (5.176/693.434)
holding_motion_gen: 	3135.1 ms/21 = 149.3 ms (75.456/356.452)
get_stable_4_3: 	0.0 ms/1 = 0.0 ms (0.005/0.005)
get_stable_4_2: 	5.2 ms/7 = 0.7 ms (0.469/0.941)

solve-mean: 1492.4 ms
solve-median: 1191.7 ms
success rate: 100.0 %


### Test without IK checkers

In [22]:
gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)

In [23]:
log_woFeas = []
for _ in range(20):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=[],
                                          tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF, max_time=10)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_woFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q87), ('AtConf', q87), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p91), ('AtPose', 4L, p91), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p91, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p92), ('AtPose', 5L, p92), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p92, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q87), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-t

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.082
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.187
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g21), sample-pose:(4, 1)->(#p23), inverse-kinematics:(4, #p23, #g21)->(#q48, #t1843), test-cfree-traj-pose:(#t1843, 5, p95)->(), inverse-kinematics:(4, p94, #g21)->(#q47, #t1842), test-cfree-traj-pose:(#t1842, 5, p95)->(), test-cfree-traj-pose:(#t1842, 4, p94)->(), test-cfree-pose-pose:(4, #p23, 5, p95)->(), test-cfree-approach-pose:(4, p94, #g21, 5, p95)->(), test-cfree-approach-pose:(4, #p23, #g21, 5, p95)->(), plan-free-motion:(#q48, q91)->(#t1884), plan-free-motion:(q91, #q47)->(#t1885), plan-holding-motion:(#q47, #q48, 4, #g21)->(#t1886)]
Action plan (5, 0.000): [move_free(q91, #q47, #t1885), pick(4, p94, #g21, #q47, #t1842), move_holding(#q47, #q48, 4, #g21, #t1886), place(4, #p23, #g21, #q48, #t1843), move_free(#q48, q91, #t1884)]
iter=0, outs=1) sample-grasp:(4)->[(g57)]
iter=0, outs=1) sample-pose

q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-2.71389979194755, -0.3047511583700157, -2.7157285208676014, -2.4840238282195815, -0.7004655246149153, 2.867543012046386, 1.7546519414316757)
go on
q_grasp: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
iter=inf, outs=0) inverse-kinematics:(4, p98, g60)->[]
iter=3, outs=1) sample-grasp:(4)->[(g61)]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-0.17874639573274542, 1.1374769517538854, 1.2038521895773393, -1.8781811832346804, -0.8239896425998763, 1.7440369187911269, 1.6726923383992254)
go on
q_grasp: (-0.17824263677035704, 1.1769515945486235, 1.1425073567821773, -1.9404756615773726, -0.8847809822390776, 1.8295088103449486, 1.7021559584471642)
obstacles: [4L, 1, 2, 3]
q_approach: (1.2344512348608219, 0.5848336302627009, -0.40251068241000093, -1.9249355941897155, 0.39381119134

iter=inf, outs=1) plan-free-motion:(q103, q104)->[(c122)]
iter=inf, outs=1) plan-free-motion:(q105, q103)->[(c123)]
iter=inf, outs=1) plan-holding-motion:(q104, q105, 4, g63)->[(c124)]
Summary: {complexity: 2, cost: 0.000, evaluations: 36, iterations: 3, length: 2, run_time: 0.508, sample_time: 0.253, search_time: 0.255, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 6926 | p_success: 0.963 | overhead: 0.001
External: sample-grasp | n: 8985 | p_success: 0.994 | overhead: 0.002
External: inverse-kinematics | n: 30210 | p_success: 0.217 | overhead: 0.048
External: plan-free-motion | n: 5910 | p_success: 0.891 | overhead: 0.126
External: plan-holding-motion | n: 2268 | p_success: 0.991 | overhead: 0.059
External: test-cfree-pose-pose | n: 3174 | p_success: 1.000 | overhead: 0.001
External: test-cfree-approach-pose | n: 6246 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 11584 | p_success: 0.937 | ov

Wrote: statistics/py2/kuka-tamp.pkl
         1590542 function calls (1526913 primitive calls) in 0.740 seconds

   Ordered by: internal time
   List reduced from 1475 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.111    0.037    0.112    0.037 pkg/planning/motion/moveit/moveit_py.py:156(plan_joint_motion_py)
    99912    0.058    0.000    0.074    0.000 /usr/lib/python2.7/pickle.py:917(load_int)
        2    0.049    0.024    0.188    0.094 /usr/lib/python2.7/pickle.py:851(load)
  50035/1    0.041    0.000    0.102    0.102 /usr/lib/python2.7/pickle.py:269(save)
   200100    0.034    0.000    0.034    0.000 {method 'read' of 'file' objects}
        2    0.033    0.016    0.033    0.016 {posix.read}
    99880    0.029    0.000    0.040    0.000 /usr/lib/python2.7/pickle.py:1184(load_append)
    49970    0.016    0.000    0.027    0.000 /usr/lib/python2.7/pickle.py:443(save_int)
   100068    0.016    0.000    0.016  

...

q_grasp: (-2.0538350424986493, -0.5830429550562455, 2.2277479746688034, -2.5583152115947767, 0.42733517934027093, 2.687002489690382, -0.5467903774847263)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.6139270739749965, -0.9596559501224461, 1.849961342962375, -2.497406409176309, 0.94285588875871, 2.291241618599881, -0.8979198406251866)
go on
q_grasp: (-1.7316038753641942, -0.9913853735542699, 2.003505928590355, -2.487059082349431, 1.0553331320964408, 2.3883094249439907, -0.9631244165346581)
obstacles: [4L, 1, 2, 3]
q_approach: (-1.5811502429518018, -1.0722248708016484, 1.821429735426037, -2.484833037732631, 1.0409482813521285, 2.2028276744654884, -0.9566268740934416)
go on
q_grasp: (-1.6781610929188406, -1.102265577488691, 1.9621046565388636, -2.475277284580131, 1.1541904740049205, 2.2911270991882415, -1.0157750072290825)
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (-1.3686334060017367, 0.9660167651607444, 1.38373227

Attempt: 1 | Results: 358 | Depth: 1 | Success: False | Time: 0.514
Attempt: 2 | Results: 715 | Depth: 1 | Success: False | Time: 1.039
Attempt: 3 | Results: 1023 | Depth: 0 | Success: True | Time: 1.877
Stream plan (37, 24, 0.001): [sample-grasp:(4)->(g100), sample-pose:(4, 2)->(p182), inverse-kinematics:(4, p182, g100)->(q287, c229), test-cfree-traj-pose:(c229, 5, p174)->(), sample-pose:(4, 1)->(p175), inverse-kinematics:(4, p175, g100)->(q275, c227), test-cfree-traj-pose:(c227, 5, p174)->(), test-cfree-traj-pose:(c229, 4, p182)->(), sample-grasp:(4)->(g102), inverse-kinematics:(4, p173, g102)->(q286, c228), test-cfree-traj-pose:(c228, 4, p173)->(), test-cfree-traj-pose:(c228, 5, p174)->(), sample-grasp:(4)->(g101), inverse-kinematics:(4, p182, g101)->(#q121, #t6117), test-cfree-traj-pose:(#t6117, 5, p174)->(), sample-pose:(4, 2)->(p184), inverse-kinematics:(4, p184, g101)->(#q119, #t6115), test-cfree-traj-pose:(#t6115, 4, p184)->(), inverse-kinematics:(4, p184, g102)->(#q120, #t6116

iter=inf, outs=1) test-cfree-traj-pose:(c232, 4, p183)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c233, 4, p185)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c237, 4, p182)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c238, 4, p185)->[()]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: None
obstacles: [4L, 1, 2, 3]
q_approach: (1.7670099777313784, -0.964890947754169, -2.0569891635825335, -2.2526552304380227, -0.5814631717087596, 2.620453213748669, 2.6748264316524852)
go on
q_grasp: (1.899787074434508, -1.017240874906355, -2.169921029466434, -2.2171850636826775, -0.6293376261911169, 2.7141250411311564, 2.712016555151987)
iter=inf, outs=1) inverse-kinematics:(4, p184, g103)->[(q317, c241)]
iter=inf, outs=1) test-cfree-traj-pose:(c241, 5, p174)->[()]
q_approach: (-1.464307500064385, -0.7558832918518746, 2.5278778

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.092
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.200
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g39), inverse-kinematics:(4, p187, #g39)->(#q124, #t6812), test-cfree-traj-pose:(#t6812, 4, p187)->(), test-cfree-traj-pose:(#t6812, 5, p188)->(), sample-pose:(4, 1)->(#p55), inverse-kinematics:(4, #p55, #g39)->(#q125, #t6813), test-cfree-traj-pose:(#t6813, 5, p188)->(), test-cfree-pose-pose:(4, #p55, 5, p188)->(), test-cfree-approach-pose:(4, p187, #g39, 5, p188)->(), test-cfree-approach-pose:(4, #p55, #g39, 5, p188)->(), plan-free-motion:(#q125, q323)->(#t6855), plan-free-motion:(q323, #q124)->(#t6856), plan-holding-motion:(#q124, #q125, 4, #g39)->(#t6854)]
Action plan (5, 0.000): [move_free(q323, #q124, #t6856), pick(4, p187, #g39, #q124, #t6812), move_holding(#q124, #q125, 4, #g39, #t6854), place(4, #p55, #g39, #q125, #t6813), move_free(#q125, q323, #t6855)]
iter=0, outs=1) sample-grasp:(4)->[(g105)]
q

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.085
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.223
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g40), inverse-kinematics:(4, p190, #g40)->(#q126, #t6871), test-cfree-traj-pose:(#t6871, 5, p191)->(), test-cfree-traj-pose:(#t6871, 4, p190)->(), sample-pose:(4, 1)->(#p56), inverse-kinematics:(4, #p56, #g40)->(#q127, #t6872), test-cfree-traj-pose:(#t6872, 5, p191)->(), test-cfree-pose-pose:(4, #p56, 5, p191)->(), test-cfree-approach-pose:(4, p190, #g40, 5, p191)->(), test-cfree-approach-pose:(4, #p56, #g40, 5, p191)->(), plan-free-motion:(q328, #q126)->(#t6913), plan-free-motion:(#q127, q328)->(#t6915), plan-holding-motion:(#q126, #q127, 4, #g40)->(#t6914)]
Action plan (5, 0.000): [move_free(q328, #q126, #t6913), pick(4, p190, #g40, #q126, #t6871), move_holding(#q126, #q127, 4, #g40, #t6914), place(4, #p56, #g40, #q127, #t6872), move_free(#q127, q328, #t6915)]
iter=0, outs=1) sample-grasp:(4)->[(g107)]
q

In [22]:
print(gtimer)

time_array = np.array(gtimer.timelist_dict["solve"])
success_array = np.array(log_woFeas)<1
print("solve-mean: {} ms".format(np.round(np.mean(time_array[np.where(success_array)[0]]), 1)))
print("solve-median: {} ms".format(np.round(np.median(time_array[np.where(success_array)[0]]), 1)))
print("success rate: {} %".format(np.mean(success_array)*100))

solve: 	114004.4 ms/20 = 5700.2 ms (944.096/10438.012)
sample_grasps_4: 	87.0 ms/20 = 4.3 ms (2.755/12.515)
get_stable_4_1: 	426.4 ms/441 = 1.0 ms (0.395/2.828)
ik_fn: 	64947.5 ms/4492 = 14.5 ms (1.353/231.827)
check_feas: 	15598.1 ms/4492 = 3.5 ms (1.345/10.35)
ReachChecker: 	4004.7 ms/4492 = 0.9 ms (0.35/3.058)
GraspChecker: 	8115.1 ms/4492 = 1.8 ms (0.59/6.77)



NameError: name 'log_woFeas' is not defined

## play plan

In [23]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.001)